<h2><center>PCIC Watershed statistics </h2></center>
This script turns PCIC datasets into rasters to use for analysis. This script has an example of making the dataset smaller prior to creating rasters for faster processing

User defined function to create rasters (function also found in myfunctions.py file)

In [4]:
def CreateRaster(lon_array,lat_array,data_array,res_lat,res_long,save_directory):
    from osgeo import gdal
    from osgeo import gdal_array
    from osgeo import osr,ogr

    xmin,ymin,xmax,ymax = [lon_array.min(),lat_array.min(),lon_array.max(),lat_array.max()]
    nrows,ncols = np.shape(data_array)
    geotransform=(xmin-res_long,res_long,0,ymin-res_lat,0, res_lat)  

    output_raster = gdal.GetDriverByName('GTiff').Create(save_directory,ncols, nrows, 1 ,gdal.GDT_Float32)  # Open the file

    output_raster.SetGeoTransform(geotransform)  # Specify its coordinates
    srs = osr.SpatialReference()                 # Establish its coordinate encoding
    srs.ImportFromEPSG(3005)                     # This one specifies WGS84 lat long.
                                                 # Anyone know how to specify the 
                                                 # IAU2000:49900 Mars encoding?
    output_raster.SetProjection( srs.ExportToWkt() )   # Exports the coordinate system                                                # to the file
    output_raster.GetRasterBand(1).WriteArray(data_array)   # Writes my array to the raster
    output_raster.FlushCache()

    return

In [5]:
from netCDF4 import Dataset,num2date
from datetime import date,datetime
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal,osr

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import show
import rasterio.mask

import fiona
import pandas as pd

from sklearn import linear_model

<h3> Defining datetime of dataset

In [6]:
Dir='C:/Users/vore/Desktop/HydroMet_Project/Data/PCIC/Temp_Max_monthly.nc'

In [7]:
nc_file=Dataset(Dir) # PCIC Data File

#---------------------------------Defining Time------------------------------------------------------------------------
t_units=nc_file.variables['time'].units   # Units of time
t_cal=nc_file.variables['time'].calendar  # Calendar Units of time

t_datetime=[]
for x in np.arange(0,len(nc_file.variables['time'])):
    t=(float(np.ma.getdata(nc_file.variables['time'][x]))) 
    t_datetime.append(num2date(t,units=t_units,calendar=t_cal)) #change to datetime format

#save time and month in array format
t_month=[];t_year=[]
for x in np.arange(0,len(t_datetime)):
    t_month.append(t_datetime[x].month)
    t_year.append(t_datetime[x].year)
t=np.array([t_year,t_month]) 

#---------------------------------------Defining Location------------------------------------------
# Defining Latitude
lat=[]
for x in np.arange(0,len(nc_file.variables['lat'])):
    lat.append(nc_file.variables['lat'][x])

#Defining Longitude
long=[]
for x in np.arange(0,len(nc_file.variables['lon'])):
    long.append(nc_file.variables['lon'][x])
    
lat=np.array(lat)
long=np.array(long)

<h3> Extract Data from .nc file </h3>
This code is written to find the monthly sums of precipitation for each grid cell. By changing the variable and np.sum function, any descriptive statistic can be found for a variable of choice on a grid-cell scale.

In [10]:
#Initate Dataframe
var_Month=pd.DataFrame({'Year':[],'Month':[],'TMin':[]})


for YEAR in np.arange(min(t_year),max(t_year)+1):
    
    #define the indices that correspond to the respected year
    index_year=np.where(t[0]==YEAR)[0]
    
    for month in np.arange(1,13,1):
        #extract var for a given year
        var_YEAR=nc_file.variables['tasmax'][min(index_year):max(index_year)+1,:,:]
        #indicies for a given month
        index_month=np.where(t[1][min(index_year):max(index_year+1)]==month)[0]
        
        #extract var values for a given month
        var_MONTH=var_YEAR[min(index_month):max(index_month)+1]
        var_MONTH=np.array(var_MONTH)
        
        #replace no data with nan values
        var_MONTH[var_MONTH<-1e+38]=np.nan
        
        #get average minimum temperature for each month
        p=np.average(np.array(var_MONTH),axis=0)  
        
        #save monthly var totals in a dataframe
        var_Month=var_Month.append({'Year':YEAR,'Month':month,'TMin':p},ignore_index=True)

In [11]:
var_Month

,Year,Month,TMin
0,1945.0,1.0,"[[-4.969068, -4.8021865, -5.0698557, -5.163528..."
1,1945.0,2.0,"[[-5.058704, -4.3978877, -4.0144286, -4.371202..."
2,1945.0,3.0,"[[-3.8166301, -2.4483054, -1.4927164, -2.43399..."
3,1945.0,4.0,"[[-0.6795798, 0.66068894, 1.4578907, 0.6261433..."
4,1945.0,5.0,"[[9.9739, 11.330113, 12.138265, 11.304742, 9.1..."
...,...,...,...
811,2012.0,8.0,"[[12.465051, 13.805534, 14.639952, 13.7535095,..."
812,2012.0,9.0,"[[8.105134, 9.393467, 10.159194, 9.356017, 7.5..."
813,2012.0,10.0,"[[-1.3066093, -0.47456944, 0.07345372, -0.6078..."
814,2012.0,11.0,"[[-8.0034075, -7.315228, -6.6278577, -6.867701..."


<h3> Cut data down to smaller extent </h3>
For faster processing I cut the Rasters down to a smaller extent (for my work this is watershed sized) before covert them into rasters. 

In [13]:
# An array of the 6 watersheds I am working with 
WS_array=['Spius','Chilko','Chilcotin','Nation','Osilinka','Mesilinka']

# Lat and Long coordinates of each of the watersheds extents in format of [min, max]
Osilinka_WS_lat=[55.7,56.8];Osilinka_WS_long=[-126.3,-124.3]
Mesilinka_WS_lat=[55.7,56.8];Mesilinka_WS_long=[-127.2,-123.5]
Nation_WS_lat=[54.8,55.8];Nation_WS_long=[-126,-123.1]
Chilko_WS_lat=[50.6,53];Chilko_WS_long=[-125.1,-122.3]
Chilcotin_WS_lat=[50.6,53];Chilcotin_WS_long=[-125.1,-122.3]
Spius_WS_lat=[49.5,50.4];Spius_WS_long=[-121.4,-120.7,]

#complile extent data into dataframe
WS_extent=pd.DataFrame({'Watershed':WS_array,'Latitude':[Spius_WS_lat,Chilko_WS_lat,Chilcotin_WS_lat,Nation_WS_lat,
                            Osilinka_WS_lat,Mesilinka_WS_lat],'Longitude':[Spius_WS_long,Chilko_WS_long,Chilcotin_WS_long,
                            Nation_WS_long,Osilinka_WS_long,Mesilinka_WS_long]})



For each watershed, year, and month, this code creates a raster, mask the raster to the watershed extent, find the average minimum temperature of the watershed for the given month, then deletes the raster. This is best used to extract watershed statistics. 

In [15]:
#Watersheds to make rasters of
WS_array=['Spius','Chilko','Chilcotin','Nation','Osilinka','Mesilinka']

#where to save raster
Save_file='C:/Users/vore/Desktop/HydroMet_Project/Data/PCIC/Working_Rasters/T_working_New.tif'

P_totals=pd.DataFrame({'WS':[],'Year':[],'Month':[],'TempMin':[]})

for WS in WS_array: 
    for year in np.arange(1979,2012):
        for month in np.arange(1,13,1):
            
            #definethe smaller extent to make a raster of
            Extent_w=WS_extent[WS_extent.Watershed==WS]

            #extract indicies that meet the extent requirements of lat and long
            lat_idx=np.where((lat<np.array(Extent_w.Latitude)[0][1]) & (lat >np.array(Extent_w.Latitude)[0][0]))[0]
            long_idx=np.where((long <np.array(Extent_w.Longitude)[0][1]) & (long > np.array(Extent_w.Longitude)[0][0]))[0]

            #extract the minimum temperature array for the given month/year
            E_working= var_Month[(var_Month.Year==year)&(var_Month.Month==month)].TMin
            idx=E_working.index[0]
            E_working=E_working[idx]

            #create array of TMin, lat, and long (within smaller extent) that will be used to create the raster
            Z=[];W=[]
            for i in lat_idx:
                for j in long_idx:
                    Z.append([E_working[i,j],lat[i], long[j]])
            
            #re-organizing datasets for Create Raster function
            Z=np.array(Z)
            res_lat= np.unique(Z[:,1])[1]-np.unique(Z[:,1])[0]
            res_long=np.unique(Z[:,2])[1]-np.unique(Z[:,2])[0]
            var_array=np.reshape(Z[:,0],(-1,len(np.unique(Z[:,2]))))
            lat_array=np.reshape(Z [:,1], (-1,len(np.unique(Z[:,2]))))
            lon_array=np.reshape(Z [:,2],(-1,len(np.unique(Z[:,2]))))

            CreateRaster(lon_array,lat_array,var_array,res_lat,res_long,Save_file)
        
            #Open the watershed shapefile to mask the raster to
            shapefile= fiona.open("C:/Users/vore/Desktop/HydroMet_Project/GIS_maps/Shapefiles_WaterFeatures/Individual_ws/%s/%s_WS.shp"%(WS,WS), "r") 
            features=[]
            for feature in shapefile:
                features.append(feature["geometry"])

            #Mask raster to watershed extent
            with rasterio.open(Save_file) as src:
                out_image, out_transform = rasterio.mask.mask(src, features,crop=True)
                out_meta = src.meta.copy()
                #show(out_image)
                out_image=np.concatenate(out_image[0])
            
            #
            idx_null=np.where(out_image==0)
            for x in idx_null:
                out_image[x]=np.nan

            #Average the minimum temperature values in the watershed 
            Sum_M=np.nanmean(out_image) ## Change this to any watershed statistic desired

            #Erase the Raster that was created (to save space)
            gdal.GetDriverByName('GTiff').Delete(Save_file)

            #Append watershed statistic to dataframe
            P_totals= P_totals.append({'WS':WS,'Year':year,'Month':month,'TempMin':Sum_M},ignore_index=True)

#export dataframe to excel
# with pd.ExcelWriter('C:/Users/vore/Desktop/HydroMet_Project/Sharable_Data/PCIC/TMax_PCIC_ALL.xlsx') as writer:  
#     P_totals.to_excel(writer, sheet_name=WS)



In [16]:
P_totals

,WS,Year,Month,TempMin
0,Spius,1979.0,1.0,-8.825379
1,Spius,1979.0,2.0,-1.685325
2,Spius,1979.0,3.0,5.570432
3,Spius,1979.0,4.0,8.740647
4,Spius,1979.0,5.0,12.834581
...,...,...,...,...
2371,Mesilinka,2011.0,8.0,14.227114
2372,Mesilinka,2011.0,9.0,11.262010
2373,Mesilinka,2011.0,10.0,4.206355
2374,Mesilinka,2011.0,11.0,-4.306751
